Here is the base code for takings paragraphes of dpef and splitting them into sentences, then keep only long sentences.

MIN_NB_OF_TOKENS=8 seems to do the trick. 

Improvements:
- exception to not split on "etc." does not work !
- Remove last filter and look at what has <8 words. Mostly fragments, mainly titles, etc. but may shows errors in parsing.
- Questions were kept here, maybe to exclude if appear in top sentences. but seem not frequent.
- main mistakes are of shape "['13 21 14 7 25 24 17 15 4 22 23']" --> add criteria to require actual words ?

In [1]:
import pandas as pd
# text
from spacy.lang.fr import French
from spacy.cli.download import download
import spacy
# download('fr_core_news_sm')

In [10]:
input_filename = "../../data/processed/DPEFs/dpef_paragraphs.csv"
output_filename = "../../data/processed/DPEFs/dpef_paragraphs_sentences.csv"
output_filename2 = "../../data/processed/DPEFs/dpef_paragraphs_sentences_long_format.csv"
df = pd.read_csv(input_filename, sep=";")
df = df[df.paragraph.notna()] # nan created while saving/ removing header -> to correct
df = df.sample(5000)

In [11]:
df.head()

,project_denomination,paragraph_id,page_nb,paragraph,x_group,y_min_paragraph,y_max_paragraph
3403,vinci,112,47,+50 %,3,676,706
8660,auchanholding,859,146,aux collaborateurs de signaler des faits de co...,0,276,435
2629,eiffage,974,176,centres de reconversion professionnelle ;,2,573,582
1752,eiffage,97,129,Les contrats à l’exportation réalisés dans d’a...,2,597,703
4611,edf,633,171,"▪ un nouveau projet en Champagne-Ardennes, « S...",2,137,149


In [6]:
# params
MIN_NB_OF_TOKENS = 8

# custom parser to split sentences while ignoring title section like cf. and splitting on "etc."
nlp = French()

def custom_sentence_boundaries(doc):
    for i, token in enumerate(doc[2:]):
        if exception_to_split(token):
            token.is_sent_start = False
        if exception_to_not_split(token):
            token.is_sent_start = True
    return doc

def exception_to_split(token):
    if 'cf' in token.nbor(-2).text and token.nbor(-1).text == ".":
        return True
    return False

def exception_to_not_split(token):
    if 'etc' in token.nbor(-2).text and token.nbor(-1).text == ".":
        return True
    return False
nlp.add_pipe(nlp.create_pipe('sentencizer')) # to keep default sentencizer as
nlp.add_pipe(custom_sentence_boundaries) # add exception and remove exceptions
# split, keep only the text and forget the nlp tagging for now
df_sent = df["paragraph"].apply(lambda x: [sent.text for sent in nlp(x).sents if len(sent)>MIN_NB_OF_TOKENS])
# save
df["paragraph_sentences"] = df_sent.values
print(df.shape)
df = df[df["paragraph_sentences"].apply(lambda x : len(x)>0)]
print(df.shape)
df.to_csv(output_filename, sep=";")

(5000, 8)
(2473, 8)


In [146]:
# reshaping following https://stackoverflow.com/questions/53860398/pandas-dataframe-how-do-i-split-one-row-into-multiple-rows-by-multi-value-colum/53860543
# convert list of pd.Series then stack it
df2 = (df
 .set_index(df.columns[:-1].values.tolist())['paragraph_sentences'] # all except last colname as index
 .apply(pd.Series)
 .stack()
 .reset_index()
 .drop('level_{}'.format(len(df.columns)-1), axis=1)
 .rename(columns={0:'sentence'}))
df2.to_csv(output_filename2, sep=";")

In [9]:
df.paragraph

4423     logements occupés par des clients particuliers...
12149    des associations telles que la Croix Rouge, Em...
13358                      D A R E 2 0 1 8 À N E W Y O R K
10938    Le pourcentage de salariés déclarés en situati...
300      Afrique, Inde, Moyen Amérique Orient Centrale ...
                               ...                        
2254     Un panel large d’actions de communication pour...
11664    liées au transport des collaborateurs et au tr...
1192     la démarche Ecosite utilisée par Bouygues Cons...
12091    3.5.1.3.6. Agir pour la santé, la sécurité et ...
9780     Les autres pays du Groupe ont également dévelo...
Name: paragraph, Length: 2473, dtype: object